In [ ]:
import os
import google.generativeai as genai


```
{
  "questions": [
    {"question": "What is inflation?", "answer": "Increase in prices over time"}
  ],
  "words": [
    {
      "word": "Loan",
      "formal_definition": "An amount of money borrowed...",
      "simple_definition": "Money you borrow and pay back with interest"
    }
  ]
}

```

In [ ]:
import streamlit as st
import google.generativeai as genai
import json
import random

# Set up Google Gen AI API
API_KEY = "your_google_api_key"
genai.configure(api_key=API_KEY)

# Example user history JSON
user_history = {
    "questions": [
        {"question": "What is inflation?", "answer": "Increase in prices over time"}
    ],
    "words": [
        {
            "word": "Loan",
            "formal_definition": "An amount of money borrowed...",
            "simple_definition": "Money you borrow and pay back with interest"
        }
    ]
}

# Function to get quiz questions with multiple choices and explanations
def get_quiz_questions(history_json, mode="personalized"):
    prompt = (
        f"Generate 5 simple finance multiple-choice questions for a beginner. "
        f"Mode: {mode}. User history:\n{json.dumps(history_json)}\n"
        "Each question should have:\n"
        "- 4 answer options (A, B, C, D) with one correct answer.\n"
        "- A short explanation of why the correct answer is right."
    )
    
    response = genai.generate_text(prompt=prompt)
    
    questions = []
    for block in response.result.split("\n\n"):  # Split by double newline
        if ":" in block:
            lines = block.strip().split("\n")
            question_text = lines[0].strip()
            options = [opt.strip() for opt in lines[1:5]]  # Extract options
            correct_answer = lines[5].split(":")[-1].strip()  # Extract answer
            explanation = lines[6].split(":")[-1].strip()  # Extract explanation
            questions.append({"question": question_text, "options": options, "answer": correct_answer, "explanation": explanation})
    
    return questions

# Streamlit UI
st.title("📊 Interactive Finance Quiz")

# Select quiz type
quiz_type = st.selectbox("Choose quiz type:", ["Personalized (based on your history)", "General Finance"])

if st.button("Generate Quiz"):
    mode = "personalized" if "Personalized" in quiz_type else "general"
    questions = get_quiz_questions(user_history, mode)

    # Store in session state
    st.session_state["questions"] = questions
    st.session_state["score"] = 0
    st.session_state["current_question"] = 0
    st.session_state["quiz_finished"] = False
    st.session_state["correct_answers"] = []  # Track correct answers
    st.session_state["wrong_answers"] = []  # Track incorrect answers

# Quiz Session
if "questions" in st.session_state and not st.session_state.get("quiz_finished", False):
    q_index = st.session_state["current_question"]
    question = st.session_state["questions"][q_index]

    st.subheader(f"Q{q_index+1}: {question['question']}")
    
    # Multiple-choice options
    user_choice = st.radio("Choose an answer:", question["options"], key=f"q{q_index}")

    if st.button("Submit Answer"):
        if user_choice == question["answer"]:
            st.success("✅ Correct!")
            st.session_state["score"] += 1
            st.session_state["correct_answers"].append(question["question"])
        else:
            st.error(f"❌ Incorrect. Correct answer: {question['answer']}")
            st.session_state["wrong_answers"].append({"question": question["question"], "explanation": question["explanation"]})
        
        st.info(f"ℹ️ Explanation: {question['explanation']}")

        if q_index + 1 < len(st.session_state["questions"]):
            st.session_state["current_question"] += 1
        else:
            st.session_state["quiz_finished"] = True

# Show final score and progress bar
if st.session_state.get("quiz_finished", False):
    st.success(f"🎉 Quiz completed! Your Score: {st.session_state['score']} / {len(st.session_state['questions'])}")
    
    st.progress(st.session_state["score"] / len(st.session_state["questions"]))

    # Show performance summary
    st.subheader("📊 Performance Review")
    
    if st.session_state["correct_answers"]:
        st.success("✅ Questions you answered correctly:")
        for correct_q in st.session_state["correct_answers"]:
            st.write(f"- {correct_q}")

    if st.session_state["wrong_answers"]:
        st.error("❌ Questions you got wrong:")
        for wrong_q in st.session_state["wrong_answers"]:
            st.write(f"- {wrong_q['question']}")
            st.info(f"📝 Explanation: {wrong_q['explanation']}")

    # Flashcards
    st.subheader("📖 Finance Flashcards")
    for item in user_history["words"]:
        with st.expander(f"📌 {item['word']}"):
            st.write(f"**Formal Definition:** {item['formal_definition']}")
            st.write(f"**Simple Definition:** {item['simple_definition']}")

